### <b>□ sigmoid 계층과 Cross entropy 계층의 계산 그래프</b>
![fig4-10](dl2_images/fig4-10.png)
$y$는 신경망이 출현한 확률, $t$는 정답 레이블 <br>
역전파 부분 $y-t$<br>
$\quad$ 정답 레이블이 $1$이라면, $y$가 $1$에 가까워질 수록 오차가 줄어든다.<br>
$\quad$ 오차가 크면 크게 학습하고, 오차가 작으면 작게 학습<br>

    예: you와 goodbye라는 맥락이 입력되었을 때 say가 타겟?
         y         -        t
        0.9               yes (1) = -0.1
                          no  (0) = 0.9

### <b>□ Negative sampling</b>                          
    지금까지는 맥락을 입력했을 때 나오는 타겟에 대하여 정답에 해당하는 긍정적인 예만 확인
        예 : 맥락 : you 와 goodbye, 타겟 : say
       
    그러면 네거티브(부정적 단어) 도 대상으로 해서 이진분류하는 학습을 해야하는데 say 외에 모든 다른 단어를 
    다 학습 시키는것은 어휘수가 늘어나면 신경망이 감당을 할 수 없으니 근사적인 해법으로 부정적인 단어를 
    샘플링하여 학습시키는것이 '네거티브 샘플링' 기법이 입니다. 
    
#### <b>16.네거티브 샘플링의 오차함수는 오차를 어떻게 계산하는가?</b>
![fig4-17](dl2_images/fig4-17.png)


#### 17. 그러면 부정 단어는 어떻게 샘플링 하나요?
![fig4-18](dl2_images/fig4-18.png)

    말뭉치의 통계데이터를 기초로해서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출해서 샘플링

### ※ 문제13. (점심시간 문제) 네거티브 샘플링을 이해하기 위한 예제. 0~9 사이의 숫자중에 무작위로 하나 출력되게 하시오

In [1]:
import numpy as np
print(np.random.choice(10))

7


### ※ 문제14. 아래의 words 리스트에서 단어 5개를 무작위로 샘플링하시오

In [3]:
import numpy as np
words = ['you','say','goodbye','I','.']

print(np.random.choice(words,size=5))

['goodbye' 'you' 'say' 'goodbye' 'I']


### ※ 문제15. 5개를 무작위로 추출하는데 중복없이 추출되게 하시오

In [4]:
import numpy as np
words = ['you','say','goodbye','I','.']

print(np.random.choice(words,size=5, replace=False))

['goodbye' '.' 'you' 'say' 'I']


### ※ 문제16. 아래의 확률 분포에 따라 샘플링되게 하시오

In [37]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]

print(np.random.choice(words, p=p, size=3, replace=False))

['I' '.' 'you']


### ※ 문제17. 책에서 네거티브 샘플링에서는 위의 확률에서 아래의 식 4.4를 따르라고 권고하고 있습니다. 확률이 낮은 단어의 확률을 살짝 높이려고 권고하고 있습니다. 테스트 해보세요
![e4-4](dl2_images/e4-4.png)

In [38]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)

print(new_p)

[0.64196878 0.33150408 0.02652714]


    수정 전 확률이 0.01인 원소가 수정 후에는 0.026으로 높아졌다.
    이처럼 낮은 확률의 단어가 좀 더 쉽게 샘플링 되도록 하기 위한 조치로 0.75제곱 수행

### ※ 문제18. 네거티브 샘플링하는 UnigramSampler 함수를 이용해서 샘플링 해보시오

In [41]:
from ch04.negative_sampling_layer import UnigramSampler


corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2


sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)

print(negative_sample)

[[2 3]
 [1 0]
 [2 4]]


## <b>4장 내용 정리</b>
    CBOW 신경망을 큰 말뭉치를 넣을 수 있도록 개선
    
    2가지 방법
        1. 입력층 --> 은닉층 
                        embedding 계층 추가
        2. 은닉층 --> 출력층, 오차함수 
                        sigmoid 함수로 출력층 함수 변경
                        negavtive sampling
                                ↓
                              cbow.py
    
    학습 시키는 코드 -> train.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [45]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.cbow import CBOW
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print(vocab_size)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성 (개선된 CBOW 생성)
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

10000
| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 2[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 4[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 6[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 8[s] | 손실 4.04
| 에폭 1 |  반복 101 / 9295 | 시간 10[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 12[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 15[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 17[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 19[s] | 손실 3.36
| 에폭 1 |  반복 201 / 9295 | 시간 21[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 23[s] | 손실 3.18
| 에폭 1 |  반복 241 / 9295 | 시간 25[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 27[s] | 손실 3.00
| 에폭 1 |  반복 281 / 9295 | 시간 29[s] | 손실 2.97
| 에폭 1 |  반복 301 / 9295 | 시간 31[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 32[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 34[s] | 손실 2.84
| 에폭 1 |  반복 361 / 9295 | 시간 36[s] | 손실 2.82
| 에폭 1 |  반복 381 / 9295 | 시간 38[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 40[s] | 손실 2.77
| 에폭 1 |  반복 421 / 9295 | 시간 42[s] | 손실 2.76
| 에폭 1 |  반복 44

| 에폭 1 |  반복 3541 / 9295 | 시간 336[s] | 손실 2.43


KeyboardInterrupt: 

### ※ 문제20. 큰 말뭉치를 CBOW 신경망에 넣어서 완성한 pickle파일을 로드하고 model에서 입력층의 가중치를 불러와서 you와 유사한 단어 top 5를 출력하시오

In [51]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

most_similar('you', word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125


### 문제21. year와 가까운 의미의 단어가 무엇인지 top3를 출력하시오

In [52]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

most_similar('year', word_to_id, id_to_word, word_vecs, top=5)


[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625


### ※ 문제22. anology 함수를 이용해서 CBOW 신경망의 밀집벡터를 가지고 단어의 유추문제를 푸시오

In [53]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)



[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slow

### ※ 문제23. do의 과거동사를 맞출 수 있는지 확인하시오

In [61]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 유추(analogy) 작업
print('-'*50)
analogy('take', 'took', 'do',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] take:took = do:?
 ca: 7.89453125
 does: 7.24609375
 did: 7.0625
 wo: 6.7578125
 could: 6.15234375


### ※ 문제24. 넣고 싶은 것을 넣고 테스트 하시오

In [64]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 유추(analogy) 작업
print('-'*50)
analogy('he', 'has', 'i',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] he:has = i:?
 had: 5.91796875
 've: 5.6953125
 have: 4.734375
 was: 4.203125
 does: 4.16015625


    정리 P.189
        1. 모델에 따라 정확도가 다르다.
        2. 일반적으로 말뭉치가 클 수록 결과가 좋다.
        3. 단어 벡터 차원수는 적당한 크기가 좋다. (너무 커도 정확도가 나쁘다)
    
    유추 문제를 이용하면 벡터의 덧셈과 뺄셈 문제를 풀 수 있다.
        king - man = women = ?